### 学习bpe基本概念和有关合并 代码实践 2024.10.21

In [2]:
# 先读入进来需要处理的数据 北京大学研究生新生手册
with open("manual.txt", "r") as f:
    text = f.read()
tokens = text.encode(encoding="utf-8") # 原始的raw_bytes
print(tokens[:100])
print(type(tokens)) # 下面进行类型转化
tokens = list(map(int, tokens))
print(f"max tokens : {max(tokens)}")
print(len(text))
print(len(tokens))

# bpe 算法当中，寻找两个词组最小合并的方法
def get_stas(ids):
    count = {}
    for pair in zip(ids, ids[1:]):
        count[pair] = count.get(pair, 0) + 1
    return count
stats = get_stas(tokens)
# print(stats)
## 按照从大到小排序
print(sorted(((v, k) for k, v in stats.items()), reverse=True))
top_pair = max(stats, key=stats.get)
top_pair

b'\xe5\x8c\x97 \xe4\xba\xac \xe5\xa4\xa7 \xe5\xad\xa6\nPEKING UNIVERSITY\n\xe5\x8c\x97 \xe4\xba\xac \xe2\xbc\xa4 \xe5\xad\xa6 \xe7\xa0\x94 \xe7\xa9\xb6 \xe2\xbd\xa3 \xe2\xbc\xbf \xe5\x86\x8c\n\xef\xbc\x88 2 0 2 3 \xe7\x89\x88 \xef\xbc\x89\n\xe5\x8c\x97 \xe4\xba\xac \xe2\xbc'
<class 'bytes'>
max tokens : 240
173983
466657
[(8084, (239, 188)), (6636, (228, 184)), (5715, (229, 173)), (5654, (173, 166)), (5408, (227, 128)), (4350, (188, 140)), (3710, (229, 133)), (3639, (231, 148)), (3548, (228, 186)), (3458, (229, 174)), (3221, (231, 154)), (3220, (154, 132)), (3016, (232, 175)), (2963, (228, 189)), (2958, (230, 156)), (2640, (128, 129)), (2559, (140, 229)), (2534, (128, 130)), (2408, (148, 159)), (2274, (132, 229)), (2242, (229, 143)), (2185, (231, 187)), (2158, (229, 138)), (2100, (230, 160)), (2001, (229, 144)), (1973, (229, 164)), (1955, (228, 187)), (1951, (229, 136)), (1918, (232, 174)), (1849, (229, 141)), (1685, (172, 229)), (1667, (230, 150)), (1665, (230, 136)), (1630, (130, 10)

(239, 188)

In [3]:
# 创建合并对
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i + 1 < len(ids) and pair[0] == ids[i] and pair[1] == ids[i + 1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i +=1
    return newids
# 一次合并减少的数量
print(f"length : {len(merge(tokens, top_pair,241))}")
print(f"一次合并减少的比例 : {round((1 - len(merge(tokens, top_pair, 241)) / len(tokens)) * 100,3)} % ")

length : 458573
一次合并减少的比例 : 1.732 % 


In [4]:
# 通过循环创建一个初始的迭代字典表
vocab_size = 280 # 最终想要的字典长度
num_merges = vocab_size - 240 # 需要合并的数量
ids = list(tokens) # 拷贝一份

merges = {}
# 迭代处理 同时我们可以同时维护一个最优的合并序列
for i in range(num_merges):
    stats = get_stas(ids)
    max_pair = max(stats, key=stats.get)
    idx = 241 + i
    ids = merge(ids, max_pair, idx)
    print(f"Merginig {max_pair} into {idx} successfully")
    merges[max_pair] = idx


Merginig (239, 188) into 241 successfully
Merginig (228, 184) into 242 successfully
Merginig (229, 173) into 243 successfully
Merginig (243, 166) into 244 successfully
Merginig (227, 128) into 245 successfully
Merginig (241, 140) into 246 successfully
Merginig (229, 133) into 247 successfully
Merginig (231, 148) into 248 successfully
Merginig (228, 186) into 249 successfully
Merginig (229, 174) into 250 successfully
Merginig (231, 154) into 251 successfully
Merginig (251, 132) into 252 successfully
Merginig (232, 175) into 253 successfully
Merginig (228, 189) into 254 successfully
Merginig (230, 156) into 255 successfully
Merginig (245, 129) into 256 successfully
Merginig (245, 130) into 257 successfully
Merginig (248, 159) into 258 successfully
Merginig (229, 143) into 259 successfully
Merginig (231, 187) into 260 successfully
Merginig (229, 138) into 261 successfully
Merginig (230, 160) into 262 successfully
Merginig (229, 144) into 263 successfully
Merginig (229, 164) into 264 succe

In [5]:
merges

{(239, 188): 241,
 (228, 184): 242,
 (229, 173): 243,
 (243, 166): 244,
 (227, 128): 245,
 (241, 140): 246,
 (229, 133): 247,
 (231, 148): 248,
 (228, 186): 249,
 (229, 174): 250,
 (231, 154): 251,
 (251, 132): 252,
 (232, 175): 253,
 (228, 189): 254,
 (230, 156): 255,
 (245, 129): 256,
 (245, 130): 257,
 (248, 159): 258,
 (229, 143): 259,
 (231, 187): 260,
 (229, 138): 261,
 (230, 160): 262,
 (229, 144): 263,
 (229, 164): 264,
 (228, 187): 265,
 (229, 136): 266,
 (232, 174): 267,
 (229, 141): 268,
 (230, 150): 269,
 (230, 136): 270,
 (257, 10): 271,
 (231, 160): 272,
 (272, 148): 273,
 (229, 175): 274,
 (254, 141): 275,
 (229, 155): 276,
 (231, 169): 277,
 (277, 182): 278,
 (273, 278): 279,
 (233, 153): 280}

In [68]:
# 看一下合并后的效果
print(len(ids))
print(f"合并减少的数量 : {round(1 - len(ids) / len(tokens),3) * 100 } %")

355198
合并减少的数量 : 23.9 %


### 接下来学习encoding 和 decoding 相关知识和实现

In [27]:
vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items(): # 扩充字典，第一次尝试的时候忽略了
    vocab[idx] = vocab[p0] + vocab[p1]

print(vocab)
# help(bytes)
# vocab
def decode(idx):
    raw_text = b"".join(vocab[id] for id in idx)
    # print(raw_text)
    text = raw_text.decode(encoding = "utf-8", errors = "replace")
    return text
decode([97,65])

## summarize : 采用utf-8解码的过程中，存在很多字符没有办法有效解码，这跟utf-8的编码特性有关
# 参考 ： https://en.wikipedia.org/wiki/UTF-8 

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

'aA'

In [9]:
def encode(text):
    # given a string, return list of integers(the tokens)
    tokens = list(text.encode(encoding="utf-8"))
    # 开始合并，合并过程中时刻选择第一个merges 里的元素进行合并
    while len(tokens) >= 2:
        stats = get_stas(tokens)
        pair = min(stats, key = lambda p : merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens
tokens = encode("h")
tokens

[104]

In [23]:
print(decode(encode("hello world"))) # 测试编码与解码是否成功
p = encode(text)
len(p)
# text2 = decode(encode(text))
# print(text2)
# print("hello world" == text2)

hello world


355198

In [29]:
decode(p) == text
# 任意案例进行测试


True

In [30]:
test_text = "2002年，博尔特在世界青年田径锦标赛男子200米项目首度展露锋芒，以16岁之龄成为赛史上最年轻的金牌得主。2004年，他在男子200米项目跑出19.93秒的成绩，以0.2秒之差打破罗伊·马丁的青少年世界纪录、成为首位跑入20秒以内的青少年选手。博尔特于2004年转职业，但因伤缺席二季比赛，直到2004年夏季奥林匹克运动会首度参赛。2007年，他在200米项目跑出19.75秒成绩，打破唐·夸里所创下的牙买加国家纪录。2008年5月，博尔特以9.72秒的成绩立下他个人的首个男子100米短跑世界纪录。他随后在2008年夏季奥林匹克运动会双双打破男子100米及200米的世界纪录：其中100米项目以9.683秒突破个人先前9.72秒的纪录，200米则以19.296秒取代麦可·强森在1996年亚特兰大夏季奥运会的19.313秒纪录。"
tests = decode(encode(test_text))
print(f"测试文本是否一样 : {test_text == tests}")

测试文本是否一样 : True


### 使用regex patterns 进行强制分割（GPT 做的尝试）

In [44]:
import regex as re
# Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
## clk100_base = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}++|\p{N}{1,3}+| ?[^\s\p{L}\p{N}]++[\r\n]*+|\s++$|\s*[\r\n]|\s+(?!\S)|\s"""
### 正则解释
### ` ?\p{L}+` + 表示匹配一个或多个字母, ?表示该字母前面可以有一个空格,\p{L}匹配 Unicode 字母字符: 用于匹配一个或多个字母组成的单词，并允许单词前有一个可选的空格
### ` ?\p{N}` N表示数字
### ` ?[^\s\p{L}\p{N}]+` 这一部分用于匹配除空白字符（\s）、字母（\p{L}）和数字（\p{N}）之外的其他字符
### `\s+(?!\S)` 匹配一个或多个空白字符。(?!\S) 是一个负向前瞻断言，表示在空白字符后不能有非空白字符。这部分的意思是匹配一个或多个空白字符，前提是它们不紧接着其他非空白字符。这主要用于捕获段落或句子的结尾部分中的空格。
### （句子中匹配多余空格，同时能）
### `\s+` 匹配一个或者多个空格
# 缺点： 大小写不敏感,对于‘这个符号也没法很好的有效识别
splits = re.findall(gpt2pat, "HElLo    world!! 123 1 12     ")
splits

['HElLo', "'", 'S', '    ', ' world', '!!', ' 123', ' 1', ' 12', '     ']

In [47]:
## tiktoken 简单介绍
import tiktoken

gpt2 = tiktoken.get_encoding("gpt2")
print(gpt2.encode("       hello world"))
gpt4 = tiktoken.get_encoding("cl100k_base")
print(gpt4.encode("     hello world"))

[220, 220, 220, 220, 220, 220, 23748, 995]
[257, 24748, 1917]


In [48]:
! wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
! wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
# 下载openai 的tiktoken分词器

--2024-10-22 13:06:51--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.97.129
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.97.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K   430KB/s    in 1.0s    

2024-10-22 13:06:53 (430 KB/s) - ‘vocab.bpe’ saved [456318/456318]

--2024-10-22 13:06:53--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 57.150.97.129
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|57.150.97.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving t

In [53]:
import os
import json
with open("encoder.json", "r") as f:
    encoder = json.load(f) # stands for vocab
with open("vocab.bpe", "r", encoding="utf-8") as f:
    bpe = f.read() # stands for merges
# 得到merge 列表
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe.split('\n')[1:]]
